In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.collocations import *
import nltk
import re
nltk.download('punkt')
import copy
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Dataset Zoo/final_csv.csv', index_col=0)

In [ ]:
df.head()

,Block,Author Name,Words per sentence,Commas per sentence,Excalimation per sentence,Questions per sentence,Salutation per sentence,He per sentence,She per sentence
0,"[' produced by david brannan.', 'html versi...",Arthur Conan Doyle,14.5785,1.1255,0.0600,0.1175,0.1205,0.1515,1.8845
1,"['if your peace should help another to peace, ...",Arthur Conan Doyle,12.6110,0.9550,0.1050,0.0820,0.0425,0.0705,1.5245
2,['and so we must cut down every expense until ...,Arthur Conan Doyle,10.4260,0.8675,0.1015,0.1230,0.1050,0.0795,1.1125
3,"['how mean of them!', 'dont let them have it, ...",Arthur Conan Doyle,15.8695,1.0500,0.0750,0.0685,0.0405,0.1200,2.2515
4,['it will be remembered that the 46th brigade ...,Arthur Conan Doyle,28.9245,1.8555,0.0035,0.0005,0.0765,0.0905,4.8365


In [ ]:
X = df.drop(['Author Name'], axis=1)
y = df['Author Name']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, shuffle= True)

In [ ]:
X_train.to_csv('X_train.csv', index = False)
X_test.to_csv('X_test.csv', index = False)
y_train.to_csv('y_train.csv', index = False)
y_test.to_csv('y_test.csv', index = False)

In [ ]:
!mv '/content/X_test.csv' '/content/drive/MyDrive/Colab Dataset Zoo/NLP Stuff/Final Final Model'
!mv '/content/y_test.csv' '/content/drive/MyDrive/Colab Dataset Zoo/NLP Stuff/Final Final Model'
!mv '/content/X_train.csv' '/content/drive/MyDrive/Colab Dataset Zoo/NLP Stuff/Final Final Model'
!mv '/content/y_train.csv' '/content/drive/MyDrive/Colab Dataset Zoo/NLP Stuff/Final Final Model'

In [ ]:
X_train.head()

,Block,Words per sentence,Commas per sentence,Excalimation per sentence,Questions per sentence,Salutation per sentence,He per sentence,She per sentence
221,"[""women of her sort between ourselves, tom don...",16.6920,1.3325,0.0800,0.0595,0.2335,0.0950,2.3210
306,['he has got some templars from ireland that s...,22.4855,1.9870,0.0250,0.1265,0.2420,0.0660,1.8860
52,"['that all dam-lies.', 'you is eulopean missio...",14.0175,1.0325,0.0620,0.0745,0.0765,0.0875,1.6560
388,"['mr. datchery, taking off his hat to give tha...",17.7715,1.7960,0.1030,0.1255,0.3375,0.1330,2.1415
11,['it might have been any one of four or five.'...,21.7280,1.3070,0.0245,0.0450,0.2745,0.0695,2.3250


In [ ]:
def get_bigrams_from_text(content1):  
  finder = BigramCollocationFinder.from_words(content1)
  # finder.nbest(bigram_measures.raw_freq, 100)
  bigram_array = []
  for k,v in finder.ngram_fd.items():
    if k[0] !='.' and k[1] != '.' and k[0] != 'project':
      bigram_array.append([k,v])
  
  bigram_array = sorted(bigram_array, key=lambda bigram_array: -bigram_array[1])
  return bigram_array

def get_trigrams_from_text(content1):  
  finder = TrigramCollocationFinder.from_words(content1)
  # finder.nbest(bigram_measures.raw_freq, 100)
  trigram_array = []
  for k,v in finder.ngram_fd.items():
    if k[0] !='.' and k[1] != '.':
      trigram_array.append([k,v])
  
  trigram_array = sorted(trigram_array, key=lambda trigram_array: -trigram_array[1])
  return trigram_array

In [ ]:
content_dict = {'Arthur Conan Doyle' : '',
                'Charles Dickens' : '',
                'H. G. Wells' : '',
                'Jonathan Swift' : '',
                'Jane Austen' : ''
                }

bigrams = copy.deepcopy(content_dict)
bigram_features = {'Arthur Conan Doyle' : [],
                   'Charles Dickens' : [],
                   'H. G. Wells' : [],
                   'Jonathan Swift' : [],
                   'Jane Austen' : []
                   }

trigrams = copy.deepcopy(content_dict)
trigram_features = copy.deepcopy(bigram_features)

In [ ]:
#From blocks to entire work corpus of an author
for myindex in X_train.index:
     content_dict[y_train[myindex]] += X_train['Block'][myindex]

In [ ]:
from nltk.tokenize import RegexpTokenizer
def spaceTokenized(corpus):
  words = corpus
  tokenizer = RegexpTokenizer(r'\w+')
  words = tokenizer.tokenize(words)
  len(words)
  return words

In [ ]:
nltk.download('stopwords')
def minusStopWords(words):
  from nltk.corpus import stopwords
  stop_words = set(stopwords.words('english')) 

  words = [w for w in words if not w in stop_words]
  len(words)
  return words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
for author in content_dict.keys():
    temp = content_dict[author]
    temp = spaceTokenized(temp)
    temp = minusStopWords(temp)
    
    bigrams[author] = get_bigrams_from_text(temp)
    trigrams[author] = get_trigrams_from_text(temp)
    
    # print('----------------------')
    # print(author)
    # print('----------------------')
    
    for i in bigrams[author][:30]:
        # print(i)
        bigram_features[author].append(i)

    print('')
    
    for i in trigrams[author][:10]:
        # print(i)
        trigram_features[author].append(i)

    print('')

In [ ]:
author = 'Jane Austen'
for i in range(len(bigram_features[author])):
   print(bigram_features[author][i],' ', i) # 29, 27, 26, 24, 23, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 8, 7, 5, 3, 2, 1, 0

[('miss', 'crawford'), 312]   0
[('mr', 'darcy'), 311]   1
[('sir', 'thomas'), 297]   2
[('mr', 'collins'), 263]   3
[('young', 'man'), 260]   4
[('mrs', 'bennet'), 257]   5
[('great', 'deal'), 235]   6
[('captain', 'wentworth'), 205]   7
[('mr', 'knightley'), 201]   8
[('dare', 'say'), 190]   9
[('mrs', 'jennings'), 186]   10
[('mr', 'elliot'), 174]   11
[('mrs', 'weston'), 171]   12
[('mr', 'elton'), 165]   13
[('mr', 'crawford'), 163]   14
[('lady', 'catherine'), 162]   15
[('mrs', 'norris'), 159]   16
[('miss', 'bennet'), 157]   17
[('sir', 'walter'), 148]   18
[('mr', 'bingley'), 148]   19
[('lady', 'russell'), 147]   20
[('mr', 'bennet'), 143]   21
[('every', 'thing'), 137]   22
[('miss', 'bingley'), 126]   23
[('mrs', 'grant'), 123]   24
[('mr', 'mrs'), 119]   25
[('lady', 'bertram'), 118]   26
[('colonel', 'fitzwilliam'), 118]   27
[('drawing', 'room'), 112]   28
[('mr', 'rushworth'), 110]   29


In [ ]:
for i in range(len(trigram_features[author])):
    print(trigram_features[author][i],' ', i) # 9, 6, 4, 3, 1, 0

[('godmersham', 'park', 'faversham'), 29]   0
[('sir', 'walter', 'elliot'), 27]   1
[('mr', 'john', 'knightley'), 26]   2
[('mr', 'frank', 'churchill'), 24]   3
[('mr', 'de', 'courcy'), 23]   4
[('park', 'faversham', 'kent'), 23]   5
[('affectionately', 'j', 'austen'), 22]   6
[('said', 'mr', 'knightley'), 22]   7
[('said', 'miss', 'crawford'), 22]   8
[('lady', 'catherine', 'de'), 21]   9


In [ ]:
#Features at this indices to be deleted as the contain character reference.
index_to_delete_bigram = {'Arthur Conan Doyle' : [18, 9, 7, 6, 3],
                          'Charles Dickens' : [29, 25, 24, 15, 14, 11, 10, 7, 1],
                          'H. G. Wells' : [19, 16, 15, 12, 11, 10, 4, 2, 1],
                          'Jonathan Swift' : [25, 17, 12, 10, 9, 8, 6, 2, 1, 0],
                          'Jane Austen' : [29, 27, 26, 24, 23, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 8, 7, 5, 3, 2, 1, 0]
                          }

index_to_delete_trigram = {'Arthur Conan Doyle' : [7, 4, 2, 0],
                          'Charles Dickens' : [9, 8, 7, 6, 4, 3, 0],
                          'H. G. Wells' : [9, 6, 5, 3, 2, 1, 0],
                          'Jonathan Swift' : [9, 6, 4, 3, 1, 0],
                          'Jane Austen' : [9, 8, 7, 6, 5, 4, 3, 2, 1]
                          }

In [ ]:
#Features at this indices to be deleted as the contain character reference.
# index_to_delete_bigram = {'Arthur Conan Doyle' : [26, 13, 12, 8, 6, 4],
#                           'Charles Dickens' : [22, 21, 16, 13, 12, 7, 5, 1],
#                           'H. G. Wells' : [21, 19, 17, 16, 12, 4, 3, 1],
#                           'Jonathan Swift' : [19, 18, 16, 14, 6, 4, 2, 1],
#                           'Jane Austen' : [27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 10, 8, 7, 5, 3, 2, 1, 0]
#                           }

# index_to_delete_trigram = {'Arthur Conan Doyle' : [9, 5, 1, 0],
#                           'Charles Dickens' : [7, 6, 5, 4, 3, 0],
#                           'H. G. Wells' : [8, 6, 5, 3, 2, 1, 0],
#                           'Jonathan Swift' : [9, 8, 5, 2],
#                           'Jane Austen' : [9, 8, 7, 6, 5, 4, 3, 2, 1]
#                           }

In [ ]:
#Delete Bigram features with character names.
for author in bigram_features.keys():
    for i in index_to_delete_bigram[author]:
      del bigram_features[author][i]

    for i in index_to_delete_trigram[author]:
      del trigram_features[author][i]

In [ ]:
#For saving and loading dictionaries
# with open('bigram_features.pickle', 'wb') as handle:
#     pickle.dump(bigram_features, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('trigram_features.pickle', 'wb') as handle:
#     pickle.dump(trigram_features, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('bigram_features.pickle', 'rb') as handle:
#     bigram_features = pickle.load(handle)

In [ ]:
#List --> Dictionary.
bigram_features_dict = {'Arthur Conan Doyle' : {},
                        'Charles Dickens' : {},
                        'H. G. Wells' : {},
                        'Jonathan Swift' : {},
                        'Jane Austen' : {}
                        }
trigram_features_dict = copy.deepcopy(bigram_features_dict)

In [ ]:
def ngram_list_to_dict(input_features, input_features_dict):
    for author in input_features.keys():
        for i in input_features[author]:
            input_features_dict[author][i[0]] = i[1]
    return input_features_dict

In [ ]:
def block_ngram_list_to_dict(input_features):
    ret_dict = {}
    for i in input_features:
        # print(input_features)
        ret_dict[i[0]] = i[1]
    return ret_dict

In [ ]:
bigram_features_dict = ngram_list_to_dict(bigram_features, bigram_features_dict)
trigram_features_dict = ngram_list_to_dict(trigram_features, trigram_features_dict)

In [ ]:
bigram_features_dict['Jane Austen'][('every', 'thing')]

137

In [ ]:
def calculate_bigram_score(given_bigrams, author_bigram_dicts):
    temp_dict = block_ngram_list_to_dict(given_bigrams)
    score = 0
    for i in author_bigram_dicts.keys():
        if i in temp_dict.keys():
            score += author_bigram_dicts[i] * temp_dict[i]
    return score

In [ ]:
def calculate_bigram_count(given_bigrams, author_bigram_dicts):
    temp_dict = block_ngram_list_to_dict(given_bigrams)
    score = 0
    for i in author_bigram_dicts.keys():
        if i in temp_dict.keys():
            score += temp_dict[i]
    return score

In [ ]:
trial = copy.deepcopy(X_train)

In [ ]:
ACD_bigram_score = []
CD_bigram_score = []
HGW_bigram_score = []
JA_bigram_score = []
JS_bigram_score = []

for myindex in trial.index:
    temp = trial['Block'][myindex]
    temp = spaceTokenized(temp)
    temp = minusStopWords(temp)

    bigrams_found = get_bigrams_from_text(temp)

    ACD_score = calculate_bigram_score(bigrams_found, bigram_features_dict['Arthur Conan Doyle'])
    CD_score = calculate_bigram_score(bigrams_found, bigram_features_dict['Charles Dickens'])
    HGW_score = calculate_bigram_score(bigrams_found, bigram_features_dict['H. G. Wells'])
    JA_score = calculate_bigram_score(bigrams_found, bigram_features_dict['Jane Austen'])
    JS_score = calculate_bigram_score(bigrams_found, bigram_features_dict['Jonathan Swift'])

    ACD_bigram_score.append(ACD_score)
    CD_bigram_score.append(CD_score)
    HGW_bigram_score.append(HGW_score)
    JA_bigram_score.append(JA_score)
    JS_bigram_score.append(JS_score)

trial['ACD_bigram_score'] = ACD_bigram_score
trial['CD_bigram_score'] = CD_bigram_score
trial['HGW_bigram_score'] = HGW_bigram_score
trial['JA_bigram_score'] = JA_bigram_score
trial['JS_bigram_score'] = JS_bigram_score

In [ ]:
ACD_trigram_score = []
CD_trigram_score = []
HGW_trigram_score = []
JA_trigram_score = []
JS_trigram_score = []

for myindex in trial.index:
    temp = trial['Block'][myindex]
    temp = spaceTokenized(temp)
    temp = minusStopWords(temp)

    trigrams_found = get_trigrams_from_text(temp)

    ACD_score = calculate_bigram_score(trigrams_found, trigram_features_dict['Arthur Conan Doyle'])
    CD_score = calculate_bigram_score(trigrams_found, trigram_features_dict['Charles Dickens'])
    HGW_score = calculate_bigram_score(trigrams_found, trigram_features_dict['H. G. Wells'])
    JA_score = calculate_bigram_score(trigrams_found, trigram_features_dict['Jane Austen'])
    JS_score = calculate_bigram_score(trigrams_found, trigram_features_dict['Jonathan Swift'])

    ACD_trigram_score.append(ACD_score)
    CD_trigram_score.append(CD_score)
    HGW_trigram_score.append(HGW_score)
    JA_trigram_score.append(JA_score)
    JS_trigram_score.append(JS_score)

trial['ACD_trigram_score'] = ACD_trigram_score
trial['CD_trigram_score'] = CD_trigram_score
trial['HGW_trigram_score'] = HGW_trigram_score
trial['JA_trigram_score'] = JA_trigram_score
trial['JS_trigram_score'] = JS_trigram_score

In [ ]:
ACD_bigram_count = []
CD_bigram_count = []
HGW_bigram_count = []
JA_bigram_count = []
JS_bigram_count = []

for myindex in trial.index:
    temp = trial['Block'][myindex]
    temp = spaceTokenized(temp)
    temp = minusStopWords(temp)

    bigrams_found = get_bigrams_from_text(temp)

    ACD_count = calculate_bigram_count(bigrams_found, bigram_features_dict['Arthur Conan Doyle'])
    CD_count = calculate_bigram_count(bigrams_found, bigram_features_dict['Charles Dickens'])
    HGW_count = calculate_bigram_count(bigrams_found, bigram_features_dict['H. G. Wells'])
    JA_count = calculate_bigram_count(bigrams_found, bigram_features_dict['Jane Austen'])
    JS_count = calculate_bigram_count(bigrams_found, bigram_features_dict['Jonathan Swift'])

    ACD_bigram_count.append(ACD_count)
    CD_bigram_count.append(CD_count)
    HGW_bigram_count.append(HGW_count)
    JA_bigram_count.append(JA_count)
    JS_bigram_count.append(JS_count)

trial['ACD_bigram_count'] = ACD_bigram_count
trial['CD_bigram_count'] = CD_bigram_count
trial['HGW_bigram_count'] = HGW_bigram_count
trial['JA_bigram_count'] = JA_bigram_count
trial['JS_bigram_count'] = JS_bigram_count

In [ ]:
trial.head()

,Block,Words per sentence,Commas per sentence,Excalimation per sentence,Questions per sentence,Salutation per sentence,He per sentence,She per sentence,ACD_bigram_score,CD_bigram_score,HGW_bigram_score,JA_bigram_score,JS_bigram_score,ACD_trigram_score,CD_trigram_score,HGW_trigram_score,JA_trigram_score,JS_trigram_score,ACD_bigram_count,CD_bigram_count,HGW_bigram_count,JA_bigram_count,JS_bigram_count
40,['i found on examination that he had had the l...,20.5050,1.4590,0.0490,0.0745,0.0550,0.0985,2.5790,39969,36609,28133,685,1263,0,0,0,0,0,87,34,73,3,13
516,['illustration one gentleman was observed sudd...,9.4445,0.9650,0.1505,0.0275,0.1515,0.0580,1.1055,14276,245780,48591,2687,390,0,1823,0,0,0,43,89,65,11,3
134,['catherine was disturbed and out of spirits b...,20.1015,1.6340,0.1120,0.1095,0.1605,0.3605,2.7750,8068,23538,9881,7704,1363,0,0,0,0,0,24,22,22,45,13
29,"['the first and second i let pass, for i hardl...",19.2810,1.6390,0.0955,0.0905,0.0685,0.0550,2.3240,29126,50850,15606,1560,1006,0,214,0,0,69,80,47,38,6,8
91,"['i asked.', 'enderby nodded.', 'and you have ...",19.9280,1.3315,0.0345,0.0765,0.1030,0.1620,2.1460,16252,107747,35663,2824,516,0,0,0,0,0,46,57,69,12,5


In [ ]:
trial.to_csv('X_train1.csv', index = False)

In [ ]:
# !mv '/content/X_test.csv' '/content/drive/MyDrive/Colab Dataset Zoo/NLP Stuff/Final Final Model'
# !mv '/content/y_test.csv' '/content/drive/MyDrive/Colab Dataset Zoo/NLP Stuff/Final Final Model'
!mv '/content/X_train1.csv' '/content/drive/MyDrive/Colab Dataset Zoo/NLP Stuff/Final Final Model'
# !mv '/content/y_train.csv' '/content/drive/MyDrive/Colab Dataset Zoo/NLP Stuff/Final Final Model'

In [ ]:
trial = copy.deepcopy(X_test)

In [ ]:
ACD_bigram_score = []
CD_bigram_score = []
HGW_bigram_score = []
JA_bigram_score = []
JS_bigram_score = []

for myindex in trial.index:
    temp = trial['Block'][myindex]
    temp = spaceTokenized(temp)
    temp = minusStopWords(temp)

    bigrams_found = get_bigrams_from_text(temp)

    ACD_score = calculate_bigram_score(bigrams_found, bigram_features_dict['Arthur Conan Doyle'])
    CD_score = calculate_bigram_score(bigrams_found, bigram_features_dict['Charles Dickens'])
    HGW_score = calculate_bigram_score(bigrams_found, bigram_features_dict['H. G. Wells'])
    JA_score = calculate_bigram_score(bigrams_found, bigram_features_dict['Jane Austen'])
    JS_score = calculate_bigram_score(bigrams_found, bigram_features_dict['Jonathan Swift'])

    ACD_bigram_score.append(ACD_score)
    CD_bigram_score.append(CD_score)
    HGW_bigram_score.append(HGW_score)
    JA_bigram_score.append(JA_score)
    JS_bigram_score.append(JS_score)

trial['ACD_bigram_score'] = ACD_bigram_score
trial['CD_bigram_score'] = CD_bigram_score
trial['HGW_bigram_score'] = HGW_bigram_score
trial['JA_bigram_score'] = JA_bigram_score
trial['JS_bigram_score'] = JS_bigram_score

In [ ]:
ACD_trigram_score = []
CD_trigram_score = []
HGW_trigram_score = []
JA_trigram_score = []
JS_trigram_score = []

for myindex in trial.index:
    temp = trial['Block'][myindex]
    temp = spaceTokenized(temp)
    temp = minusStopWords(temp)

    trigrams_found = get_trigrams_from_text(temp)

    ACD_score = calculate_bigram_score(trigrams_found, trigram_features_dict['Arthur Conan Doyle'])
    CD_score = calculate_bigram_score(trigrams_found, trigram_features_dict['Charles Dickens'])
    HGW_score = calculate_bigram_score(trigrams_found, trigram_features_dict['H. G. Wells'])
    JA_score = calculate_bigram_score(trigrams_found, trigram_features_dict['Jane Austen'])
    JS_score = calculate_bigram_score(trigrams_found, trigram_features_dict['Jonathan Swift'])

    ACD_trigram_score.append(ACD_score)
    CD_trigram_score.append(CD_score)
    HGW_trigram_score.append(HGW_score)
    JA_trigram_score.append(JA_score)
    JS_trigram_score.append(JS_score)

trial['ACD_trigram_score'] = ACD_trigram_score
trial['CD_trigram_score'] = CD_trigram_score
trial['HGW_trigram_score'] = HGW_trigram_score
trial['JA_trigram_score'] = JA_trigram_score
trial['JS_trigram_score'] = JS_trigram_score

In [ ]:
ACD_bigram_count = []
CD_bigram_count = []
HGW_bigram_count = []
JA_bigram_count = []
JS_bigram_count = []

for myindex in trial.index:
    temp = trial['Block'][myindex]
    temp = spaceTokenized(temp)
    temp = minusStopWords(temp)

    bigrams_found = get_bigrams_from_text(temp)

    ACD_count = calculate_bigram_count(bigrams_found, bigram_features_dict['Arthur Conan Doyle'])
    CD_count = calculate_bigram_count(bigrams_found, bigram_features_dict['Charles Dickens'])
    HGW_count = calculate_bigram_count(bigrams_found, bigram_features_dict['H. G. Wells'])
    JA_count = calculate_bigram_count(bigrams_found, bigram_features_dict['Jane Austen'])
    JS_count = calculate_bigram_count(bigrams_found, bigram_features_dict['Jonathan Swift'])

    ACD_bigram_count.append(ACD_count)
    CD_bigram_count.append(CD_count)
    HGW_bigram_count.append(HGW_count)
    JA_bigram_count.append(JA_count)
    JS_bigram_count.append(JS_count)

trial['ACD_bigram_count'] = ACD_bigram_count
trial['CD_bigram_count'] = CD_bigram_count
trial['HGW_bigram_count'] = HGW_bigram_count
trial['JA_bigram_count'] = JA_bigram_count
trial['JS_bigram_count'] = JS_bigram_count

In [ ]:
trial.head()

,Block,Words per sentence,Commas per sentence,Excalimation per sentence,Questions per sentence,Salutation per sentence,He per sentence,She per sentence,ACD_bigram_score,CD_bigram_score,HGW_bigram_score,JA_bigram_score,JS_bigram_score,ACD_trigram_score,CD_trigram_score,HGW_trigram_score,JA_trigram_score,JS_trigram_score,ACD_bigram_count,CD_bigram_count,HGW_bigram_count,JA_bigram_count,JS_bigram_count
40,['i found on examination that he had had the l...,20.5050,1.4590,0.0490,0.0745,0.0550,0.0985,2.5790,39969,36609,28133,685,1263,0,0,0,0,0,87,34,73,3,13
516,['illustration one gentleman was observed sudd...,9.4445,0.9650,0.1505,0.0275,0.1515,0.0580,1.1055,14276,245780,48591,2687,390,0,1823,0,0,0,43,89,65,11,3
134,['catherine was disturbed and out of spirits b...,20.1015,1.6340,0.1120,0.1095,0.1605,0.3605,2.7750,8068,23538,9881,7704,1363,0,0,0,0,0,24,22,22,45,13
29,"['the first and second i let pass, for i hardl...",19.2810,1.6390,0.0955,0.0905,0.0685,0.0550,2.3240,29126,50850,15606,1560,1006,0,214,0,0,69,80,47,38,6,8
91,"['i asked.', 'enderby nodded.', 'and you have ...",19.9280,1.3315,0.0345,0.0765,0.1030,0.1620,2.1460,16252,107747,35663,2824,516,0,0,0,0,0,46,57,69,12,5


In [ ]:
trial.to_csv('X_test1.csv', index = False)

In [ ]:
!mv '/content/X_test1.csv' '/content/drive/MyDrive/Colab Dataset Zoo/NLP Stuff/Final Final Model'